In [2]:
import yfinance as yf
import numpy as np
import pandas as pd
import datetime as dt
from pandas.tseries.offsets import MonthEnd # This enables us to jump from one month to the other OR subtract a month from a date

In [3]:
df= pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")[0] # indexing for 0 otherwise a series data would be 
                                                                            # returned

In [4]:
df.head()

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,66740,1902
1,AOS,A. O. Smith,reports,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
3,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981


In [5]:
tickers=df.Symbol.to_list()
tickers

['MMM',
 'AOS',
 'ABT',
 'ABBV',
 'ABMD',
 'ACN',
 'ATVI',
 'ADM',
 'ADBE',
 'ADP',
 'AAP',
 'AES',
 'AFL',
 'A',
 'AIG',
 'APD',
 'AKAM',
 'ALK',
 'ALB',
 'ARE',
 'ALGN',
 'ALLE',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AMD',
 'AEE',
 'AAL',
 'AEP',
 'AXP',
 'AMT',
 'AWK',
 'AMP',
 'ABC',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'ANTM',
 'AON',
 'APA',
 'AAPL',
 'AMAT',
 'APTV',
 'ANET',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'AZO',
 'AVB',
 'AVY',
 'BKR',
 'BLL',
 'BAC',
 'BBWI',
 'BAX',
 'BDX',
 'WRB',
 'BRK.B',
 'BBY',
 'BIO',
 'TECH',
 'BIIB',
 'BLK',
 'BK',
 'BA',
 'BKNG',
 'BWA',
 'BXP',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BRO',
 'BF.B',
 'CHRW',
 'CDNS',
 'CZR',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CARR',
 'CTLT',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'CE',
 'CNC',
 'CNP',
 'CDAY',
 'CERN',
 'CF',
 'CRL',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CTXS',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',
 'CMC

In [6]:
# Using the tickers list, we now download the price data from yfinance

df2=yf.download(tickers, start="2009-12-31", end="2020-12-31")

[*********************100%***********************]  505 of 505 completed

4 Failed downloads:
- BF.B: No data found for this date range, symbol may be delisted
- CEG: Data doesn't exist for startDate = 1262214000, endDate = 1609369200
- BRK.B: No data found, symbol may be delisted
- OGN: Data doesn't exist for startDate = 1262214000, endDate = 1609369200


In [28]:
# We only need the Adj close price, so lets subset that column

price= df2["Adj Close"]
price.head()

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2009-12-30,20.304356,4.591151,38.750172,6.499639,NaN,21.836216,8.90,19.807898,32.673073,37.180000,...,38.512669,14.028231,42.948311,32.995991,NaN,18.858959,53.170151,28.389999,11.120522,NaN
2009-12-31,20.284763,4.562869,38.249386,6.471692,NaN,21.637022,8.73,19.608143,32.524155,36.779999,...,38.204304,13.877813,42.586090,32.634106,NaN,18.730413,52.591827,28.350000,11.146583,NaN
2010-01-04,20.434929,4.496877,38.154900,6.572423,NaN,22.101797,8.74,19.778837,32.970871,37.090000,...,41.963718,13.786258,43.185623,32.782581,NaN,18.794687,53.401501,28.670000,11.580979,NaN
2010-01-05,20.212959,5.005958,37.928116,6.583786,NaN,21.944115,8.53,19.619041,33.174648,37.700001,...,44.515926,13.622750,43.354244,32.392876,NaN,18.730413,55.091969,28.620001,11.989312,NaN
2010-01-06,20.141132,4.798553,38.258839,6.479064,NaN,21.736618,8.40,19.727999,33.527321,37.619999,...,43.932011,13.648916,43.728970,32.606281,NaN,18.596512,55.074165,28.400000,13.031859,NaN


In [46]:
mtl_ret=price.pct_change().resample("M").agg(lambda x: (x+1).prod()-1)
mtl_ret.head(12)

# The code above is doing the following things:

        # First, it is calculating the daily returns from the daily adjusted price data 
        # Second, it is sort of creating breaks in between the months. Think of it like this: there are many boxes being created
            # and each of the boxes are given the label of datetime
        # Third, the lambda function is denoting x as whatever result we get from "price.pct_change()"
            # Basically, x = daily returns. The daily returns are then multiplied to each other within the boxes of months.

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2009-12-31,-0.000965,-0.006160,-0.012923,-0.004300,0.0,-0.009122,-0.019101,-0.010085,-0.004558,-0.010759,...,-0.008007,-0.010722,-0.008434,-0.010968,0.0,-0.006816,-0.010877,-0.001409,0.002343,0.0
2010-01-31,-0.097843,0.097108,-0.025445,-0.088597,0.0,0.045646,-0.092783,-0.012287,-0.012289,-0.121805,...,0.062683,-0.020735,-0.055140,-0.046630,0.0,-0.013728,-0.047200,-0.079365,0.478566,0.0
2010-02-28,0.122368,0.380414,0.034220,0.065396,0.0,0.031652,0.276515,0.025311,-0.024884,0.072755,...,0.027311,0.001444,0.015428,-0.013123,0.0,-0.014323,0.017933,0.094636,-0.022124,0.0
2010-03-31,0.093134,0.002728,0.028856,0.148470,0.0,0.031384,0.020772,-0.029477,0.049537,0.020779,...,0.192858,0.030484,0.030462,0.055622,0.0,0.136714,0.032618,0.036052,0.177994,0.0
2010-04-30,0.054376,-0.038095,0.075859,0.111021,0.0,0.066736,-0.065891,-0.020672,0.049417,-0.050042,...,0.163656,0.025944,0.011794,0.050745,0.0,0.114615,0.028885,-0.018581,0.315476,0.0
2010-05-31,-0.107557,0.248939,0.147671,-0.016125,0.0,0.016526,0.011411,-0.070367,-0.140238,-0.045238,...,-0.046313,-0.057932,-0.101806,-0.115143,0.0,-0.034653,-0.081760,-0.053356,-0.166077,0.0
2010-06-30,-0.121446,-0.024915,-0.029368,-0.020827,0.0,0.015026,-0.007179,-0.016401,0.030117,-0.176122,...,-0.090725,0.017816,-0.056070,-0.076229,0.0,-0.046642,-0.033614,-0.077455,-0.099374,0.0
2010-07-31,-0.017588,0.260163,0.066760,0.022741,0.0,-0.056063,0.145661,0.058909,0.025614,0.086644,...,0.149600,0.066958,0.045733,0.003678,0.0,0.065350,-0.019611,0.081592,0.028744,0.0
2010-08-31,-0.034372,-0.166821,0.017560,-0.055005,0.0,-0.087253,-0.185753,0.005298,-0.076690,-0.035515,...,-0.078176,0.014552,-0.002463,-0.074617,0.0,0.009685,-0.109832,0.043003,-0.169521,0.0


In [44]:
# Calculating the past 11 month cumulated return for each month. This means we gotta use the rolling returns feauture

past_11= (mtl_ret + 1).rolling(11).apply(np.prod)-1

# Here, the first 10 entries should have NaN values because for the it is only at the 11th month that our calculation really 
# starts

In [45]:
past_11.head(12)

,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2009-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-02-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-05-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-06-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-07-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-08-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Defining the portfolio formation date

In [ ]:
# This means on this date, we are buying the winners and shorting the losers

